**<font color = black size=6>实验五:决策树</font>**

**<font color = blue size=4>第一部分:函数介绍</font>**

In [81]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

1)Counter类的使用

In [82]:
x=np.array([[0,133,1],[0,132,0],[0,133,0]])
#使用Counter类对数组第2列进行遍历
counter = Counter(x[:,1])
#第2列中有1个132和2个133，输出该counter对象可以统计这列的数值情况，便于之后的统计
print(counter)
#因为第2列中没有为0的值，所以返回0
print(counter[0])
#因为第2列中有2个133，所以返回2
print(counter[133])
#一般的字典操作方法都能在该类中使用，例如可以通过values函数返回该列的非重复值的个数，方便对某列的非重复值的个数进行查看
print(counter.values())
#可以输出所有非重复值
print(list(counter))

Counter({133: 2, 132: 1})
0
2
dict_values([2, 1])
[133, 132]


2)使用numpy中的unique实现非重复值的提取

In [83]:
x=np.array([[0,133,1],[0,132,0],[0,133,0]])
a=np.unique(x[:])
a1=np.unique(x[:,1])
print(a,a1)

[  0   1 132 133] [132 133]


**<font color = blue size=4>第二部分:实验任务</font>**

**<font color = blue size=3>1) 任务一</font>**

任务一要求完成不同量化标准(信息增益, 信息增益率, 基尼系数)下的结点划分函数

<span style="color:purple">该数据集(train_titanic.csv)为分类数据集，为泰坦尼克号的部分乘客信息以及最后是否生还。包括了四个属性特征以及一个标签(即为Survived,代表是否生还),属性特征分别为Sex(性别)，sibsp(堂兄弟妹个数)，Parch(父母与小孩的个数)，Pclass(乘客等级)  
其中该数据集无缺失值和异常值，且所有连续变量已自动转换为离散变量,标签(Survived)也自动转变为离散变量0和1，所以你无需进行数据预处理，可以直接使用该数据集。</span>

In [84]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

<span style="color:purple">1) 使用pandas库将训练数据集'train_titanic.csv'载入到Dataframe对象中</span>

In [85]:
#----your code here
train_dataframe = pd.read_csv('train_titanic.csv')

<span style="color:purple">2) 编写函数，给定任何标签数组计算其信息熵  
    输入：标签数组  
    输出：该数组对应的信息熵  
    计算信息熵公式:
某数组包含K个不同的取值，样本为第k(k=1,2,...,K)个值的数量所占比例为p_k,则其信息熵为$$Ent=-\sum_{k=1}^K p_k log_2 p_k$$</span>
    
    
<span style="color:purple">例:  
    输入:[[0],[1]]   
    输出:(-$\frac{1}{2}$ $log_2$ $\frac{1}{2}$)+(-$\frac{1}{2}$ $log_2$ $\frac{1}{2}$)=1</span>

In [86]:
def calculate_entropy(label):
    label = label.reshape(-1,1) # 设定形状
    total_count = len(label) # 标签的总数
    label_counts = Counter(label[:,0]) # 统计每个标签的数量
    ent = 0 # 初始化信息熵为0
    for count in label_counts.values():
        p_k = count / total_count # 计算标签的比例
        ent += -p_k * math.log2(p_k) # 累加信息熵
    return ent


<span style="color:purple">3) 编写函数，函数功能为将所给的数据集按照指定维度的特征进行划分为若干个不同的数据集  
    【输入】：特征集合，标签集合，指定维度  
    【输出】：划分后所得到的子树属性集合，子树标记集合</span>

<span style="color:purple">例子:  
    【输入】:特征集合:[[0,0,0],[0,0,1],[1,0,2]]  
    标签集合:[[0],[1],[2]]  
    指定维度:0</span>  
    
<span style="color:purple">【输出】:[[0,0,0],[0,0,1]]和[[1,0,2]]  
    [[0],[1]]和[[2]]  
    tips:即将特征按其第0维度进行划分，由于第0维特征有0和1两个不同的数值，所以特征集合划分为[[0,0,0],[0,0,1]]和[[1,0,2]]，标签集合划分为[[0],[1]]和[[2]]</span>

In [87]:
def split_dataset(feature, label, d):    
    # 获取该维度的所有不重复的值
    unique_values = np.unique(feature[:, d])

    # 初始化空的列表来保存划分后的数据
    split_feature = []
    split_label = []

    # 根据unique_values的值，划分feature和label
    for value in unique_values:
        # 对于等于 unique_value 的子集
        matched_indices = feature[:, d] == value # 匹配则对应位置的索引为True
        # 根据匹配值来确定是否加入相应的矩阵行
        subset_split_feature = feature[matched_indices]
        subset_split_label = label[matched_indices]
        # 添加结果
        split_feature.append(subset_split_feature)
        split_label.append(subset_split_label)
    # 转换为矩阵
    split_feature = np.array(split_feature,dtype=object)
    split_label = np.array(split_label,dtype=object)
    # 返回划分结果
    return split_feature, split_label


<span style="color:purple">4) 编写函数，函数功能为进行【一次】决策树的结点划分，遍历找出该特征集合中信息增益(使用ID3算法中的公式计算)【最大】的特征  
    输入：特征集合，标签集合  
    输出：该次划分的最佳信息增益值，最佳划分维度  
    计算信息增益公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征feature,该特征包含V个不同的取值，其中值为第v(v=1,2,...,V)个值的样本数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该特征值对应的信息增益为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$
该函数中你需要计算出每个特征的信息增益并输出，然后返回最佳的信息增益值和对应的特征的维数</span>

In [88]:
def one_split_ID3(x_data, y_label):
    best_entropy = float('-inf') # 初始化最佳信息增益值为无穷小
    best_dimension = -1 # 初始化最佳维度
    base_entropy = calculate_entropy(y_label) # 计算基础信息熵
    
    # 遍历每个划分特征维度
    for d in range(x_data.shape[1]):
        # 按当前维度划分数据
        _, split_labels = split_dataset(x_data, y_label, d) 
        # 计算特征划分后的信息熵
        new_entropy = sum([len(sub_labels) / len(y_label) * calculate_entropy(sub_labels) for sub_labels in split_labels]) 
        # 计算信息增益值
        info_gain = base_entropy - new_entropy 
        # 若当前的信息增益值更大,进行更新
        if info_gain > best_entropy:
            best_entropy = info_gain # 更新最佳信息增益值
            best_dimension = d # 更新最佳划分维度
            
    # 返回最佳的信息增益值和对应的特征的维度
    return best_entropy, best_dimension

<span style="color:purple">5) 编写函数，函数功能为进行【一次】决策树的结点划分，遍历找出该特征集合中信息增益率(使用C4.5算法中的公式计算)【最大】的特征  
    输入：特征集合，标签集合  
    输出：最佳划分的信息增益率值，对应的划分维度  
    计算信息增益率公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征类型feature,该特征包含V个不同的取值，值为第v(v=1,2,...,V)个值的样本数量为|$D^v$|$(\sum_{v=1}^V|D^v|=|D|)$,该特征本身的信息熵为Ent(feature),则该特征值对应的信息增益率为$$Gain\_ratio(D,feature)=\frac{Gain(D,feature)}{Ent(feature)}$$
该函数中你需要输出每个特征的信息增益率，之后返回最佳的信息增益率和对应的特征维数</span>

In [89]:
def one_split_C4_5(x_data, y_label):
    best_gain_ratio = float('-inf')  # 初始化最佳信息增益率为无穷小
    best_dimension = -1  # 初始化最佳维度为-1
    base_entropy = calculate_entropy(y_label)  # 计算基础信息熵
    
    # 遍历每一个特征维度
    for d in range(x_data.shape[1]):
        # 按当前维度划分数据
        _, split_labels = split_dataset(x_data, y_label, d) 
        # 计算按当前维度划分后的新信息熵
        new_entropy = sum([len(sub_labels) / len(y_label) * calculate_entropy(sub_labels) for sub_labels in split_labels])
        # 计算信息增益
        info_gain = base_entropy - new_entropy
        # 计算划分特征本身的信息熵
        ent_feature = calculate_entropy(x_data[:,d])
        # 计算信息增益率
        gain_ratio = info_gain/ent_feature
        # 如果当前的信息增益率更大，则进行更新
        if gain_ratio > best_gain_ratio:
            best_gain_ratio = gain_ratio # 更新最佳信息增益率
            best_dimension = d # 更新最佳划分维度
            
    # 返回最佳信息增益率和对应的特征维度
    return best_gain_ratio, best_dimension

<span style="color:purple">6) 编写函数，进行【一次】决策树的结点划分，遍历找出该特征集合中基尼系数(使用CART算法中的公式计算)最小的特征以及最佳的划分值  
    输入：特征集合，标签集合  
    输出：最佳的基尼系数，对应的划分维度，最佳划分值  
    计算基尼系数公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,该集合中标签类别总共有K类，第k类样本所占比例为$p_k$(k=1,2,..,K),则该数据集对应的基尼系数为$$Gini(D)=1-\sum_{k=1}^K{p_k}^2$$  
    而取其中一个特征feature，选定该特征的一个值value，根据该特征的值是否为value将数据集分为两个子集$D_1$和$D_2$,则该特征对应的基尼系数为$$Gini\_index(D,feature)=\frac{|D_1|}{|D|}Gini(D_1)+\frac{|D_2|}{|D|}Gini(D_2)$$
该函数中你需要遍历每一列特征，找出每列中的非重复值，计算出每个非重复值的基尼系数，返回最小的基尼系数、对应的特征维数和非重复值（分类值）。</span>

In [90]:
# 定义计算基尼系数的函数
def gini(D):
    # 获取数据集D的总长度
    total_counts = len(D)
    # 使用Counter统计每个标签的频率
    label_counts = Counter(D[:,0]) # 注意输入数据形状
    # 计算基尼系数
    gini_index = 1
    # 遍历样本的全部类别
    for count in label_counts.values():
        p_k = count / total_counts # 计算所占比例
        gini_index -= p_k**2  
    # 返回基尼系数
    return gini_index


def one_split_CART(x_data, y_label):
    best_gini_index = float('inf') # 初始化最小的基尼系数为无穷大
    best_dimension = -1 # 初始化最佳特征维度
    best_value = None # 初始化分类值
    
    # 遍历每一列特征
    for d in range(x_data.shape[1]):
        # 找出每列中的非重复值
        unique_values = np.unique(x_data[:,d])
        # 选取特征的一个值value
        for value in unique_values:
            # 根据特征值是否为value将数据集划分为D1和D2
            D1 = y_label[x_data[:, d] == value]
            D2 = y_label[x_data[:, d] != value]
            # 计算基尼系数
            gini_index = len(D1) / len(y_label) * gini(D1) + len(D2) / len(y_label) * gini(D2) 
            # 若新的基尼系数更小,则进行更新
            if gini_index < best_gini_index:
                best_gini_index = gini_index # 更新最佳基尼系数
                best_dimension = d # 更新最佳划分特征维度
                best_value = value  # 获取该特征值
                
    # 返回最小的基尼系数、对应的特征维度和最佳划分值            
    return best_gini_index, best_dimension, best_value


<span style="color:purple">7) 应用之前你在第4、5、6个部分编写的三个函数，在训练数据集'train_titanic.csv'上依次使用这些函数进行【一次】结点划分，并输出对应的最佳特征维数以及相应的信息增益值/信息增益率/(基尼系数和分类值)</span>

In [91]:
#----your code here
train_dataframe = pd.read_csv('train_titanic.csv')
train_data = np.array(train_dataframe)
# 将数据进行格式转换 转换成矩阵
x_data = train_data[:,:-1]
y_label = train_data[:,-1].reshape(-1,1)

# 1. 使用one_split_ID3函数进行一次节点划分
best_entropy, best_dimension1= one_split_ID3(x_data, y_label)
print("one_split_ID3函数")
print(f"最佳信息增益值: {best_entropy}")
print(f"最佳划分维度: {best_dimension1}")
print()

# 2. 使用one_split_C4_5函数进行一次节点划分
best_gain_ratio, best_dimension2= one_split_C4_5(x_data, y_label)
print("one_split_C4_5函数")
print(f"最佳信息增益率: {best_gain_ratio}")
print(f"最佳划分维度: {best_dimension2}")
print()

# 3. 使用one_split_CART函数进行一次节点划分
best_gini_index, best_dimension3, best_value = one_split_CART(x_data, y_label)
print("one_split_CART函数")
print(f"最佳基尼系数: {best_gini_index}")
print(f"最佳划分维度: {best_dimension3}")
print(f"最佳划分值: {best_value}")
print()


one_split_ID3函数
最佳信息增益值: 0.10750711887455178
最佳划分维度: 0

one_split_C4_5函数
最佳信息增益率: 0.11339165967945304
最佳划分维度: 0

one_split_CART函数
最佳基尼系数: 0.29649157246415725
最佳划分维度: 0
最佳划分值: 0



**<font color = blue size=3>2) 任务二</font>**

任务二承接任务一，用【ID3】算法实现一棵完整的决策树。

<span style="color:purple">1) 整理任务一的代码，根据实验二要求做适当修改来编写函数，【从剩余的特征集A中】寻找使得信息增益最大的特征   
    输入：数据集D、剩余的特征集A    
    输出：最佳划分的特征维数    
    计算信息增益公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征feature,该特征包含V个不同的取值，特征值为第v(v=1,2,...,V)个值的样本数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该特征对应的信息增益为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$</span>

In [92]:
def best_split(D, A):
    best_entropy = float('-inf')  # 初始化最佳信息增益值为无穷小
    best_dimension = -1  # 初始化最佳维度 (可省略)
    base_entropy = calculate_entropy(D[:, -1])  # 数据集D的标签是最后一列 计算基础信息熵
    
    features = D[:, :-1] # 特征值
    labels = D[:, -1] # 标签值
    # 对于剩余的每个特征
    for d in A:
        # 按当前维度划分数据
        _, split_labels = split_dataset(features, labels, d) 
        # 计算特征划分后的信息熵
        new_entropy = np.sum([len(sub_labels) / len(labels) * calculate_entropy(sub_labels) for sub_labels in split_labels]) 
        # 计算信息增益值
        info_gain = base_entropy - new_entropy 
        # 若当前的信息增益值更大,则进行更新
        if info_gain > best_entropy:
            best_entropy = info_gain # (可省略)
            best_dimension = d # 更新最佳划分特征维度
            
    # 返回最佳的特征维度
    return best_dimension

<span style="color:purple">2) 完成DTree类中的TreeGenerate、train函数以完成决策树的构建。并完成DTree类中的predict函数来用构建好的决策树来对测试数据集进行预测并输出预测准确率。</span>

In [93]:
# 树结点类
class Node:
    def __init__(self, isLeaf=True, label=-1, feature_index=-1):
        self.isLeaf = isLeaf # isLeaf表示该结点是否是叶结点
        self.label = label # label表示该叶结点的label（当结点为叶结点时有用）
        self.feature_index = feature_index # feature_index表示该分支结点的划分特征的序号（当结点为分支结点时有用）
        self.children = {} # children表示该结点的所有孩子结点，dict类型，方便进行决策树的搜索
        
    def addNode(self, val, node):
        self.children[val] = node #为当前结点增加一个划分特征的值为val的孩子结点

In [94]:
# 决策树类
class DTree:
    def __init__(self):
        self.tree_root = None #决策树的根结点
        self.possible_value = {} # 用于存储每个特征可能的取值
    
        
 
    '''
    TreeGenerate函数用于递归构建决策树,伪代码参照课件中的"Algorithm 1 决策树学习基本算法"
     
    '''    
    
    def TreeGenerate(self, D, A):
        
        # 生成结点 node
        node = Node()
        
        # if D中样本全属于同一类别C then
        #     将node标记为C类叶结点并返回
        # end if

        unique_labels = np.unique(D[:, -1]) # 获取非重复的样本值
        # 所有样本同一类别
        if len(unique_labels) == 1:
            node.isLeaf = True # 标记为叶子节点
            node.label = unique_labels[0] # 标记label
            return node
       
        # if A = Ø OR D中样本在A上取值相同 then
        #     将node标记叶结点，其类别标记为D中样本数最多的类并返回
        # end if
        
        if len(A) == 0 or np.all([len(np.unique(D[:, feature])) == 1 for feature in A]):
            node.isLeaf = True  # 标记为叶子结点
            node.label = Counter(D[:, -1]).most_common(1)[0][0] # 标记label为D中样本数最多的类
            return node
        
        # 从A中选择最优划分特征a_star
        # （选择信息增益最大的特征，用到上面实现的best_split函数） 
        a_star = best_split(D,A) # 选择信息增益最大的特征
        node.feature_index = a_star
        node.isLeaf = False # 标记为非叶子结点

        # for a_star 的每一个值a_star_v do
        #     为node 生成每一个分支；令D_v表示D中在a_star上取值为a_star_v的样本子集
        #     if D_v 为空 then
        #         将分支结点标记为叶结点，其类别标记为D中样本最多的类
        #     else
        #         以TreeGenerate(D_v,A-{a_star}) 为分支结点
        #     end if
        # end for

        for a_star_v in self.possible_value[a_star]:
            child_node = Node() # 初始化分支结点
            D_v = D[D[:, a_star] == a_star_v] # 生成D_v
            
            if len(D_v) == 0:
                child_node.isLeaf = True # 标记为叶子结点
                child_node.label = Counter(D[:,-1]).most_common(1)[0][0] # 标记类别为D中样本最多的类
            else:
                child_node = self.TreeGenerate(D_v, A - {a_star})
            # 添加分支结点
            node.addNode(a_star_v, child_node)   
             
        return node

        
    
    '''
    train函数可以做一些数据预处理(比如Dataframe到numpy矩阵的转换,提取属性集等),并调用TreeGenerate函数来递归地生成决策树
 
    '''
    
    def train(self, D):
#         D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
#         A = set(range(D.shape[1] - 1)) # 特征集A
        
#         #记下每个特征可能的取值
#         for every in A:
#             self.possible_value[every] = np.unique(D[:, every])
        
#         self.tree_root = self.TreeGenerate(D, A) # 递归地生成决策树，并将决策树的根结点赋值给self.tree_root

        D = np.array(D) 
        A = set(range(D.shape[1] - 1))        
        
        for every in A:
            self.possible_value[every] = np.unique(D[:, every])
        
        self.tree_root = self.TreeGenerate(D, A) 
    
        return
     
    '''
    predict函数对测试集D进行预测, 并输出预测准确率（预测正确的个数 / 总数据数量）
    
    '''
    
    def predict(self, D):
        
#         D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）      
#         #对于D中的每一行数据d，从当前结点x=self.tree_root开始，当当前结点x为分支结点时，

#         #则搜索x的划分特征为该行数据相应的特征值的孩子结点（即x=x.children[d[x.index]]），不断重复，
#         #直至搜索到叶结点，该叶结点的标签就是数据d的预测标签
        
        D = np.array(D) # 转换为numpy矩阵
        correct = 0 # 初始化预测准确的个数
        # 遍历每一行
        for i in range(D.shape[0]):
            x = self.tree_root # 从当前节点x=self.tree_root开始
            d = D[i, :-1] # D中每一行数据d
            # 直至搜索到叶子结点
            while not x.isLeaf:
                x=x.children[d[x.feature_index]]
            # 当到达叶节点，比较预测标签和真实标签
            if x.label == D[i,-1]:
                correct += 1 # 预测正确 累加预测正确的个数
        acc = correct / D.shape[0]  # 计算准确率(预测正确的个数/总数据数量)

        return acc


In [95]:
#----your code here
test_dataframe = pd.read_csv('test_titanic.csv')
# 构建决策树
tree = DTree()
tree.train(train_dataframe)

# train=np.array(train_dataframe)
# print(train.shape)
# train_labels=train[:,-1]
# print(Counter(train_labels))


# 利用构建好的决策树对测试数据集进行预测，输出预测准确率（预测正确的个数 / 总数据数量）
accuracy = tree.predict(test_dataframe)
print("准确率:",accuracy)


准确率: 0.8415841584158416


In [96]:
#展示生成的决策树结构
def display_tree(node, indent=''):
    if node.isLeaf:
        print(indent + "Leaf Node: label =", node.label)
    else:
        print(indent + "Branch Node: feature_index =", node.feature_index)
        for value, child in node.children.items():
            print(indent + "|--> Child Value:", value)
            display_tree(child, indent + "   ")


display_tree(tree.tree_root)



Branch Node: feature_index = 0
|--> Child Value: 0
   Branch Node: feature_index = 2
   |--> Child Value: 0
      Branch Node: feature_index = 3
      |--> Child Value: 1
         Branch Node: feature_index = 1
         |--> Child Value: 0
            Leaf Node: label = 0
         |--> Child Value: 1
            Leaf Node: label = 0
         |--> Child Value: 2
            Leaf Node: label = 0
         |--> Child Value: 3
            Leaf Node: label = 0
         |--> Child Value: 4
            Leaf Node: label = 0
         |--> Child Value: 5
            Leaf Node: label = 0
         |--> Child Value: 8
            Leaf Node: label = 0
      |--> Child Value: 2
         Branch Node: feature_index = 1
         |--> Child Value: 0
            Leaf Node: label = 0
         |--> Child Value: 1
            Leaf Node: label = 0
         |--> Child Value: 2
            Leaf Node: label = 0
         |--> Child Value: 3
            Leaf Node: label = 0
         |--> Child Value: 4
            

**<font color = blue size=4>第三部分:作业提交</font>**

<span style="color:purple">本次实验分两周完成,实验报告提交地址(学号+姓名+实验五):https://send2me.cn/s0Yr-w4z/QCKd4ibz34miqQ   截止日期:10.20 14:20</span>

<span style="color:purple">实验课件获取地址:https://www.jianguoyun.com/p/DZRe9cwQp5WhChjz4p8FIAA</span>